# (MIDI) Prédiction et génération de fichier midi

Lien du sujet : https://github.com/neuronalX/Reservoir-Jupyter
Apprendre le reservoir computing : https://github.com/neuronalX/Reservoir-Jupyter
Code d'exemple reservoir computing : https://github.com/neuronalX/FunkyReservoir

Import des modules

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
from ipywidgets import *
from IPython.display import *

Classe pour générer le réseau :

In [6]:
class Network(object):

    def __init__(self, trainLen=2000, testLen=2000, initLen=100, data="MackeyGlass_t17.txt") :
        self.initLen = initLen
        self.trainLen = trainLen
        self.testLen = testLen
        self.data = np.loadtxt(data)
        self.inSize = self.outSize = 1 #Dimension de l'entrée et de la sortie
        self.resSize = 300 #Taille du réservoir (prédiction)
        #self.resSize = 1000 #Taille du réservoir (géneration)
        self.a = 0.3 #Taux de fuite alpha 
        self.spectral_radius = 1.25 #Rayon spectral
        self.input_scaling = 1. #Echelle de l'input
        self.reg =  1e-8 #None #Coefficient de régularisation - si None,
        #on utilisera le pseudo-inverse plutôt que la régression de crête

        self.mode = 'prediction'
        #self.mode = 'generative'

        #Modification de la première valeur pseudo-aléatoire : les performances du réservoir
        #devraient être dans la moyenne après 20 instances aléatoires (pour les mêmes paramètres)
        seed = self.set_seed()
        
        #Poids
        self.Win = (np.random.rand(self.resSize,1+self.inSize)-0.5) * self.input_scaling
        self.W = np.random.rand(self.resSize,self.resSize)-0.5 

        #Matrices
        #Matrice des états collectés (1,u,x) au cours du temps
        self.X = np.zeros((1+self.inSize+self.resSize,self.trainLen-self.initLen))
        #Matrice de sortie cible
        self.Ytarget = self.data[None,self.initLen+1:self.trainLen+1]

        #Vecteur des états du réservoir
        self.x = np.zeros((self.resSize,1))  
        
    def set_seed(self):
        """Making the seed (for random values) variable if None"""

        if seed is None:
            import time
            seed = int((time.time()*10**6) % 4294967295)
        try:
            np.random.seed(seed)
            print("Seed used for random values:", seed)
        except:
            print("!!! WARNING !!!: Seed was not set correctly.")
        return seed

nw = Network(trainLen=2000, testLen=2000, initLen=100, data="MackeyGlass_t17.txt")

Seed used for random values: 166609793


Récupération des données

In [3]:
import mido

print("Chargement du fichier MIDI...")

mid = mido.MidiFile('samples/house_of_the_rising_sunasax_as.mid')

print("Chargement du fichier MIDI terminé")

Chargement du fichier MIDI...
Chargement du fichier MIDI terminé


Extraction des données importantes

In [13]:
notes = []
i = 0
sumTime = 0

for msg in mid.play():
    sumTime += msg.time
    
    if(msg.type == "note_on"):
        notes.append((msg.note, sumTime))
        
    elif(msg.type == "note_off"):
        j = len(notes) - 1
        while(j >= 0):
            note, time = notes[j]
            if(note == msg.note):
                notes[j] = (note, sumTime - time)
                break
            j -= 1
        
    
    
    
    
    if(msg.type == "note_on"):
        print("noteON : " + str(msg.note) + ", time : " + str(msg.time) + ", velocity : " + str(msg.velocity), ", channel : " + str(msg.channel))
    elif(msg.type == "note_off"):
        print("-- noteOFF : " + str(msg.note) + ", time : " + str(msg.time) + ", velocity : " + str(msg.velocity), ", channel : " + str(msg.channel))
    
    
    if(i >= 100):
        break
    
    i += 1

noteON : 55, time : 6.3888825, velocity : 56 , channel : 1
-- noteOFF : 55, time : 0.2777775, velocity : 0 , channel : 1
noteON : 60, time : 0, velocity : 71 , channel : 1
-- noteOFF : 60, time : 1.11111, velocity : 0 , channel : 1
noteON : 60, time : 0, velocity : 58 , channel : 1
noteON : 62, time : 0.2777775, velocity : 65 , channel : 1
-- noteOFF : 60, time : 0.0043402734375, velocity : 0 , channel : 1
-- noteOFF : 62, time : 0.25824626953125, velocity : 0 , channel : 1
noteON : 63, time : 0.01519095703125, velocity : 64 , channel : 1
-- noteOFF : 63, time : 1.11111, velocity : 0 , channel : 1
noteON : 63, time : 0, velocity : 61 , channel : 1
noteON : 65, time : 0.2777775, velocity : 65 , channel : 1
-- noteOFF : 63, time : 0.0043402734375, velocity : 0 , channel : 1
-- noteOFF : 65, time : 0.25824626953125, velocity : 0 , channel : 1
noteON : 67, time : 0.01519095703125, velocity : 61 , channel : 1
-- noteOFF : 67, time : 0.8333325, velocity : 0 , channel : 1
noteON : 65, time : 

Affichage des notes enregistrées

In [12]:
for note in notes:
    print("Note " + str(note[0]) + " pendant " + str(note[1]) + " secondes")

Note 55 pendant 0.2777775 secondes
Note 60 pendant 1.11111 secondes
Note 60 pendant 0.28211777343749844 secondes
Note 62 pendant 0.2625865429687497 secondes
Note 63 pendant 1.11111 secondes
Note 63 pendant 0.28211777343749844 secondes
Note 65 pendant 0.2625865429687497 secondes
Note 67 pendant 9.999989999999997 secondes


Création du fichier MIDI de sortie

In [24]:
from mido import Message, MidiFile, MidiTrack

mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

for note in notes:
    track.append(Message('note_on', note=note[0], time=0))
    track.append(Message('note_off', note=note[0], time=int(note[1]*500)))

mid.save('new_song.mid')